<a href="https://colab.research.google.com/github/Tripaloski1212/Mi-primer-repositorio/blob/main/Sim_PlantasV14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ==============================================================================
# 1. BASES DE DATOS (ADN Y GEOGRAFÍA)
# ==============================================================================
BIBLIOTECA_ESPECIES = {
    '1': {'Nombre': 'Maíz', 'T_BASE': 10.0, 'T_OPT_MIN': 18.0, 'T_OPT_MAX': 32.0, 'RUE': 1.6, 'HI': 0.50, 'N_DEMANDA': 0.8, 'A': 240, 'k': 0.012},
    '2': {'Nombre': 'Trigo', 'T_BASE': 4.0, 'T_OPT_MIN': 12.0, 'T_OPT_MAX': 24.0, 'RUE': 1.1, 'HI': 0.42, 'N_DEMANDA': 0.4, 'A': 95, 'k': 0.018},
    '3': {'Nombre': 'Tomate', 'T_BASE': 10.0, 'T_OPT_MIN': 15.0, 'T_OPT_MAX': 28.0, 'RUE': 1.3, 'HI': 0.65, 'N_DEMANDA': 0.6, 'A': 150, 'k': 0.014},
    '4': {'Nombre': 'Soja', 'T_BASE': 10.0, 'T_OPT_MIN': 20.0, 'T_OPT_MAX': 30.0, 'RUE': 1.1, 'HI': 0.35, 'N_DEMANDA': 0.2, 'A': 100, 'k': 0.015},
    '5': {'Nombre': 'Girasol', 'T_BASE': 7.0, 'T_OPT_MIN': 18.0, 'T_OPT_MAX': 33.0, 'RUE': 1.2, 'HI': 0.30, 'N_DEMANDA': 0.5, 'A': 180, 'k': 0.013},
    '6': {'Nombre': 'Arroz', 'T_BASE': 12.0, 'T_OPT_MIN': 22.0, 'T_OPT_MAX': 32.0, 'RUE': 1.3, 'HI': 0.45, 'N_DEMANDA': 0.6, 'A': 110, 'k': 0.016},
    '7': {'Nombre': 'Papa', 'T_BASE': 7.0, 'T_OPT_MIN': 14.0, 'T_OPT_MAX': 22.0, 'RUE': 1.2, 'HI': 0.80, 'N_DEMANDA': 0.7, 'A': 60, 'k': 0.020}
}

GEOGRAFIAS = {
    '1': {'Tipo': 'Tropical', 'TMAX': 32, 'TMIN': 24, 'Osc': 4},
    '2': {'Tipo': 'Desértico', 'TMAX': 42, 'TMIN': 12, 'Osc': 15},
    '3': {'Tipo': 'Templado', 'TMAX': 26, 'TMIN': 14, 'Osc': 8},
    '4': {'Tipo': 'Mediterráneo', 'TMAX': 30, 'TMIN': 16, 'Osc': 10}
}

def validar(p, d, mi, ma):
    while True:
        try:
            v = input(f"{p} [{d}]: ")
            if not v: return d
            v = float(v)
            if mi <= v <= ma: return v
            print(f"Rango: {mi}-{ma}")
        except: print("Número inválido")

# ==============================================================================
# 2. CONFIGURACIÓN INTERACTIVA (SUELO, CLIMA Y PLANTA)
# ==============================================================================
def setup():
    print("\n" + "="*40 + "\n⚙️ CONFIGURACIÓN V14\n" + "="*40)
    for k, v in BIBLIOTECA_ESPECIES.items(): print(f"{k}. {v['Nombre']}")
    p_sel = input("Especie: ")
    esp = BIBLIOTECA_ESPECIES.get(p_sel, BIBLIOTECA_ESPECIES['1']).copy()

    print("\n--- SELECCIÓN GEOGRÁFICA ---")
    for k, v in GEOGRAFIAS.items(): print(f"{k}. {v['Tipo']}")
    g_sel = input("Geografía: ")
    geo = GEOGRAFIAS.get(g_sel, GEOGRAFIAS['3'])

    print("\n--- PARÁMETROS DE SUELO (MANUAL) ---")

    esp['CC'] = validar("Capacidad de Campo (mm)", 150.0, 50, 400)
    esp['PMP'] = validar("Punto Marchitez (mm)", 50.0, 10, esp['CC']-10)
    esp['N_INI'] = validar("Nitrógeno Inicial (kg/ha)", 100.0, 0, 500)
    esp['DIAS'] = int(validar("Días totales", 100, 10, 300))

    return esp, geo

# ==============================================================================
# 3. MOTOR POR BLOQUES HORARIOS Y ALERTAS
# ==============================================================================
class MotorFinal:
    def __init__(self, c, g):
        self.c, self.g = c, g
        self.h, self.n = c['CC']*0.8, c['N_INI']
        self.altura = 0.0; self.biomasa = 0.0; self.gdd = 0.0; self.dia = 0
        self.salud_f = 1.0

    def simular_dia(self):
        self.dia += 1
        # Modelo Sinusoidal por horas
        t_horas = [self.g['TMAX'] - (self.g['Osc'] * (1 - np.sin(np.pi * h / 12))) for h in range(24)]

        # Estrés térmico (Calidad del aire por horas)
        ft = np.mean([1.0 if self.c['T_OPT_MIN'] <= t <= self.c['T_OPT_MAX'] else 0.4 for t in t_horas])

        # Estrés hídrico y nutricional
        fw = max(0.1, min(1.0, (self.h - self.c['PMP']) / (self.c['CC'] - self.c['PMP'])))
        fn = max(0.2, min(1.0, self.n / 40.0))

        f_total = min(fw, fn, ft) * self.salud_f

        # Crecimiento
        gdd_dia = max(0, np.mean(t_horas) - self.c['T_BASE'])
        self.gdd += gdd_dia
        h_pot = self.c['A'] * np.exp(-4.0 * np.exp(-self.c['k'] * self.gdd))
        delta_h = max(0, (h_pot - self.altura) * f_total)
        self.altura += delta_h
        self.biomasa += 22 * (self.altura/self.c['A']) * self.c['RUE'] * f_total

        # Desgaste de recursos
        self.n = max(0, self.n - (delta_h * self.c['N_DEMANDA']))
        self.h = max(self.c['PMP'], min(self.c['CC'], self.h + np.random.choice([0,12,0]) - 5))

        return {'Día': self.dia, 'Salud': f_total, 'Eh': fw, 'En': fn, 'Et': ft, 'N': self.n, 'H': self.h}

def jugar():
    esp, geo = setup()
    sim = MotorFinal(esp, geo)
    hist = []
    auto = False

    for d in range(esp['DIAS']):
        res = sim.simular_dia()
        hist.append(res)

        if not auto:
            print(f"\n[DÍA {d}] {esp['Nombre']} en {geo['Tipo']}")
            print(f"Altura: {round(sim.altura,1)}cm | Salud: {round(res['Salud']*100)}% | N: {round(sim.n,1)}")

            # GESTIÓN ACTIVA (ALERTAS)
            if res['Eh'] < 0.5:
                if input("⚠️ Suelo Seco. ¿Regar? (s/n): ").lower() == 's': sim.h = esp['CC']
            if res['En'] < 0.4:
                if input("🧪 Falta Nitrógeno. ¿Fertilizar? (s/n): ").lower() == 's': sim.n += 50

            # Evento Aleatorio de Plaga (Influenciado por Geografía)
            prob_plaga = 0.05 if geo['Tipo'] == 'Tropical' else 0.02
            if np.random.random() < prob_plaga:
                print("🐛 ¡PLAGA DETECTADA!")
                if input("¿Fumigar? (s/n): ").lower() != 's': sim.salud_f *= 0.8

            op = input("\n[Enter] Día | [A] Auto | [F] Cosechar: ").upper()
            if op == 'A': auto = True
            if op == 'F': break

    df = pd.DataFrame(hist)
    print(f"\n--- RESULTADO FINAL: {round(sim.biomasa * esp['HI'], 2)} kg/ha ---")
    graficar(df, esp, geo)

def graficar(df, c, g):
    plt.style.use('seaborn-v0_8-darkgrid')
    fig, axs = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
    fig.suptitle(f"Simulación V14: {c['Nombre']} ({g['Tipo']})")

    axs[0].plot(df['Día'], df['Salud']*100, color='red', label='Vitalidad %')
    axs[0].legend()

    axs[1].plot(df['Día'], df['N'], color='green', label='N (kg/ha)')
    ax2 = axs[1].twinx(); ax2.plot(df['Día'], df['H'], color='blue', label='Humedad (mm)')
    axs[1].legend(loc=2); ax2.legend(loc=1)

    axs[2].plot(df['Día'], df['Et'], color='purple', label='Eficiencia Térmica (Horas)')
    axs[2].set_ylim(0, 1.1); axs[2].legend()

    plt.tight_layout(); plt.show()

if __name__ == "__main__":
    jugar()